# **thread_stack**

- thread_stack은 각 스레드의 스택 크기를 지정하는 MySQL 시스템 변수
  - 즉 커넥션당 할당되는 값이므로 마냥 크게만 잡을수는 없다.
- 스택은 스레드가 지역 변수, 함수 매개변수, 반환 주소를 저장, 지역 변수, 함수 호출 정보, 제어 흐름 데이터를 저장하고 기타 작업을 수행하는 데 사용하는 메모리 영역.
- Thread stack의 사이즈가 너무 작으면, 복잡한 쿼리 문장이나 프로시져의 재귀적 함수의 처리를 제한하게 될 수 있음.

- 가이드에는 다음과 같이 설정 권장.
  - 32bit : 192K
  - 64bit : 256K
- 값이 작으면 stack 할당을 못해서 MySQL Server가 핸들링 할 수 있는 다음의 복잡한 SQL구문을 처리하지 못함.
  - stored procedure의 recursive depth 나 다른 메모리 잡아먹는 동작 등..

thread_stack 변수는 명령줄을 사용하여 구성하거나 시작 시 구성 파일을 사용하여 설정할 수 있음

Command Line Configuration:

```
mysqld> set global thread_stack = XX
```

- XX를 데이터베이스 요구 사항에 맞는 스레드 처리 전략으로 변경.

<br>
- 변수가 변경되었는지 확인하려면 다음을 수행.

```
mysqld> show global variables like ‘thread_stack’
```

Configuration File:
```
[mysqld]
thread_stack variable = XX
```

- XX를 데이터베이스 요구 사항에 맞는 스레드 처리 전략으로 변경.
- 적용된 변경 사항을 보려면 MySQL 서버를 다시 시작해야 함.

# **thread_stack - 사용법**

-  스택은 LIFO 원리에 따라 작동.
  -  이는 스택에 배치된 마지막 항목(또는 데이터 조각)이 가장 먼저 제거되거나 액세스되는 항목임을 의미.

**왜 중요 함?**

- MySQL의 스레드는 작업을 수행할 때마다 스택을 사용하여 프로세스를 관리.
- 기본 작업의 경우 일반적으로 기본 스택 크기가 적합.
-  스레드가 복잡한 쿼리 실행이나 광범위한 저장 프로시저 실행과 같이 더 복잡한 작업을 수행하는 경우 올바르게 작동하려면 더 많은 스택 공간이 필요.

**부적절한 스택 크기로 인한 잠재적인 문제**
  - 수행 중인 작업에 비해 thread_stack 크기가 너무 낮게 설정되면 다음과 같은 몇 가지 문제가 발생할 수 있음.

  - 스레드 충돌
    - 스레드에 스택 오버플로가 발생하여 충돌이 발생할 수 있습니다. 이로 인해 데이터가 손실되거나 트랜잭션이 불완전해질 수 있음.

  - 서버 불안정성
    - 다중 스레드 충돌로 인해 전체 서버가 불안정해지고 모든 사용자와 프로세스에 영향을 미칠 수 있음.

  - 성능 저하
    - 스레드가 스택 제한에 도달하면 속도가 느려지고 쿼리 실행 시간이 길어질 수 있음.

# **hread_stack - 고려 사항**

- thread_stack 변수의 사용 및 조정을 고려할 때 시스템 성능에 대한 잠재적 영향을 이해하는 것이 중요.

**메모리 사용량**
- thread_stack 크기를 늘리면 각 스레드에서 사용하는 메모리도 늘어남.
- 특히 동시 연결이 많은 환경에서는 증가된 스택 크기를 수용할 수 있을 만큼 서버에 충분한 메모리가 있는지 확인.

**심층 재귀**
- 작업에 재귀 함수 또는 자신을 여러 번 호출하는 저장 프로시저가 포함된 경우 각 재귀는 더 많은 스택 공간을 소비.

**대규모 지역 변수**
- 대규모 지역 변수 또는 배열을 선언하고 사용하는 작업은 사용 가능한 스택 공간을 빠르게 소모할 수 있음.

**복잡한 조인 및 하위 쿼리**
- 여러 조인, 하위 쿼리 또는 파생 테이블이 포함된 쿼리는 효율적으로 처리하기 위해 더 많은 스택 메모리가 필요할 수 있음.

**오류 로그 **
- 스택 관련 오류 또는 경고에 대해 MySQL 오류 로그를 모니터링합니다. 이는 thread_stack 크기가 충분하지 않다는 표시일 수 있음.

**성능 영향**
- thread_stack 크기를 늘리면 충돌을 방지할 수 있지만 너무 높게 설정하면 메모리가 낭비되고 서버 성능이 저하될 수 있음.